In [29]:
import json
import requests
import time
from bs4 import BeautifulSoup
import pymysql
import datetime

def epi_info(series_id, state):
    epi_url = 'https://webtoon.kakao.com/content/text/' + str(series_id)
    epi_res = requests.get(epi_url)
    today_weekday = datetime.datetime.now().strftime("%a").lower()

    soup = BeautifulSoup(epi_res.content, 'html.parser')
    result = soup.select_one('#__NEXT_DATA__').get_text()
    json_data = json.loads(result)

    toon_num = series_id
    N_toon = json_data['props']['initialState']['content']['contentMap'][str(series_id)]['title']
    Toon_category_list = '#' + json_data['props']['initialState']['content']['contentMap'][str(series_id)]['genre']
    Toon_category = '#'.join([gerne for gerne in Toon_category_list.split('/')])
    Toon_link = f"https://webtoon.kakao.com/content/{N_toon}/{series_id}"
    Toon_update = 1 if day == today_weekday else 0
    Toon_platform = "3"  # 플랫폼 정보 (카카오 웹툰을 임의로 3으로 설정)
    Toon_likes = "0" # 0으로 초기화
    
    # 이미지 링크 가져오기
    Toon_imagelink = soup.find('meta', property='og:image')['content']
    Toon_newupdate = "0"
    
    return [
        toon_num,
        N_toon,
        Toon_category,
        Toon_link,
        Toon_update,
        Toon_platform,
        Toon_likes,
        Toon_imagelink,
        Toon_newupdate
    ]

def insert_into_mysql(data, cursor):
    # MySQL 서버 연결 설정 (호스트, 사용자, 비밀번호, 데이터베이스)
    host = "localhost"
    user = "root"
    password = "1234"
    database = "toon_plus"
    try:
        connection = pymysql.connect(host=host, user=user, password=password, db=database, charset='utf8', cursorclass=pymysql.cursors.DictCursor)
        cursor = connection.cursor()
        

        # 데이터베이스에 데이터 삽입
        insert_query = "INSERT INTO toon_table (toon_num, N_toon, Toon_category, Toon_link, Toon_update, Toon_platform, Toon_likes, Toon_imagelink, Toon_newupdate) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)"
        cursor.execute(insert_query, data)
        connection.commit()
        print("데이터가 성공적으로 삽입되었습니다.")
    except pymysql.Error as error:
        print(f"MySQL 오류: {error}")
    finally:
        if connection:
            connection.close()

def get_series_by_day(day):
    # 요일별 연재작 수집
    url = f'https://gateway-kw.kakao.com/section/v1/timetables/days?placement=timetable_{day}'

    res = requests.get(url)
    contents = res.json()['data'][0]['cardGroups'][0]['cards']

    error_page = []

    for series in contents:
        series_id = series['content']['id']

        try:
            data = epi_info(series_id, day)
            insert_into_mysql(data, cursor)
            time.sleep(3)

        except Exception as e:
            print(f"에러 발생: {e}")
            print(series_id)
            error_page.append(series_id)

    print(f"{day}에 대한 에러 목록: {error_page}")

def update_toon_newupdate(cursor):
    try:
        connection = pymysql.connect(host="localhost", user="root", password="1234", db="toon_plus", charset='utf8', cursorclass=pymysql.cursors.DictCursor)
        cursor = connection.cursor()
        
        # DB에서 모든 웹툰 데이터를 가져옵니다.
        cursor.execute("SELECT toon_num, N_toon FROM toon_table")
        webtoons = cursor.fetchall()

        # 웹툰 이름을 가져와서 DB에 저장된 N_toon과 비교하여 Toon_newupdate를 업데이트합니다.
        for webtoon in webtoons:
            toon_num = webtoon['toon_num']
            N_toon = webtoon['N_toon']
            new_toon_url = f'https://webtoon.kakao.com/original-webtoon?tab=new'
            
            res = requests.get(new_toon_url)
            soup = BeautifulSoup(res.content, 'html.parser')
            
            # 웹툰 이름이 웹툰 목록 페이지에 있는지 확인합니다.
            if N_toon in str(soup):
                # 웹툰이 목록에 있는 경우 Toon_newupdate를 1로 업데이트합니다.
                update_query = "UPDATE toon_table SET Toon_newupdate = 1 WHERE toon_num = %s"
                cursor.execute(update_query, toon_num)
                connection.commit()
                print(f"{N_toon}의 Toon_newupdate가 업데이트되었습니다.")
            else:
                print(f"{N_toon}은(는) 목록에 없습니다.")
        
    except pymysql.Error as error:
        print(f"MySQL 오류: {error}")
    finally:
        if connection:
            connection.close()

try:
    connection = pymysql.connect(host="localhost", user="root", password="1234", db="toon_plus", charset='utf8', cursorclass=pymysql.cursors.DictCursor)
    cursor = connection.cursor()
    cursor.execute("delete from toon_table")  # 기존 데이터 삭제
    connection.commit()

    # 월요일부터 일요일까지 반복하여 웹툰 정보 수집
    days_of_week = ['mon']
    for day in days_of_week:
        get_series_by_day(day)

    # Toon_newupdate 값을 업데이트합니다.
    update_toon_newupdate(cursor)

except pymysql.Error as error:
    print(f"MySQL 연결 오류: {error}")
finally:
    if 'connection' in locals():
        cursor.close()
        connection.close()


데이터가 성공적으로 삽입되었습니다.
데이터가 성공적으로 삽입되었습니다.
데이터가 성공적으로 삽입되었습니다.
데이터가 성공적으로 삽입되었습니다.
데이터가 성공적으로 삽입되었습니다.
데이터가 성공적으로 삽입되었습니다.
데이터가 성공적으로 삽입되었습니다.
데이터가 성공적으로 삽입되었습니다.
데이터가 성공적으로 삽입되었습니다.
데이터가 성공적으로 삽입되었습니다.
데이터가 성공적으로 삽입되었습니다.
데이터가 성공적으로 삽입되었습니다.
데이터가 성공적으로 삽입되었습니다.
데이터가 성공적으로 삽입되었습니다.
데이터가 성공적으로 삽입되었습니다.
데이터가 성공적으로 삽입되었습니다.
데이터가 성공적으로 삽입되었습니다.
데이터가 성공적으로 삽입되었습니다.
데이터가 성공적으로 삽입되었습니다.
데이터가 성공적으로 삽입되었습니다.
데이터가 성공적으로 삽입되었습니다.
데이터가 성공적으로 삽입되었습니다.
데이터가 성공적으로 삽입되었습니다.
데이터가 성공적으로 삽입되었습니다.
데이터가 성공적으로 삽입되었습니다.
데이터가 성공적으로 삽입되었습니다.
데이터가 성공적으로 삽입되었습니다.
데이터가 성공적으로 삽입되었습니다.
데이터가 성공적으로 삽입되었습니다.
데이터가 성공적으로 삽입되었습니다.
데이터가 성공적으로 삽입되었습니다.
데이터가 성공적으로 삽입되었습니다.
데이터가 성공적으로 삽입되었습니다.
데이터가 성공적으로 삽입되었습니다.
데이터가 성공적으로 삽입되었습니다.
데이터가 성공적으로 삽입되었습니다.
데이터가 성공적으로 삽입되었습니다.
데이터가 성공적으로 삽입되었습니다.
데이터가 성공적으로 삽입되었습니다.
데이터가 성공적으로 삽입되었습니다.
데이터가 성공적으로 삽입되었습니다.
데이터가 성공적으로 삽입되었습니다.
데이터가 성공적으로 삽입되었습니다.
데이터가 성공적으로 삽입되었습니다.
데이터가 성공적으로 삽입되었습니다.
데이터가 성공적으로 삽입되었습니다.
데이터가 성공적으로 삽입되었습니다.
데이터가 성공적으로 삽입되었습니다.
데이터가 성공적으로 삽입되었습니다.
데이터가 성공적으로 삽입되었습니다.
